In [1]:
import pandas as pd
data = pd.read_csv(r'data\archive\medium_articles.csv')
data.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [2]:
data.shape

(192368, 6)

In [3]:
import random
import re

def get_random_sentence_from_text(text, number_sentences):
    """
    Selects a random number of sentences from the given text.

    Parameters:
    text (str): The input text containing multiple sentences.
    number_sentences (int): The number of random sentences to select.

    Returns:
    list: A list containing the selected random sentences.
    """
    # Split text into sentences using regex to handle different punctuation marks
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    # Ensure number_sentences is not greater than available sentences
    number_sentences = min(number_sentences, len(sentences))

    # Select random sentences
    random_sentences = random.sample(sentences, number_sentences)
    
    return sentences,  random_sentences

# Example usage
text_sample = data['text'].unique()[10][:1000]
sent , random_sentences = get_random_sentence_from_text(text_sample, 2)

random_sentences

['If you’re a freelancer, everyone is curious about how you got started.',
 'In other words:\n\nYour origin story helps people decide whether or not to trust you.']

In [4]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

def draw_text_with_strikethrough(draw, text, font, x, y, fill=0):
    """
    Draw the given text in black-and-white at (x, y),
    then draw a strikethrough line across the middle.
    In mode "L", 'fill=0' is black, 'fill=255' is white.
    """
    # Draw the text first
    draw.text((x, y), text, fill=fill, font=font)
    
    # Use textbbox (Pillow 8.0.0+) to get the exact bounding box
    left, top, right, bottom = draw.textbbox((x, y), text, font=font)

    # Compute the vertical center
    line_y = (top + bottom) / 2

    # Draw the strikethrough line in black (width=3 for visibility)
    draw.line((left, line_y, right, line_y), fill=fill, width=3)

def driver(text, id):
    # ---------------------------
    # 1. Create a blank image in grayscale ("L")
    # ---------------------------
    img_width, img_height = 1024, 1024
    # In grayscale mode "L", 255 is white, 0 is black
    image = Image.new("L", (img_width, img_height), 255)
    draw = ImageDraw.Draw(image)

    # ---------------------------
    # 2. Define a font
    # ---------------------------
    try:
        font = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        # Fallback to default font if arial.ttf isn't found
        font = ImageFont.load_default()

    # ---------------------------
    # 3. Prepare text
    # ---------------------------
    text_block = text
    sentences , strike_list = get_random_sentence_from_text(text_block, 1)
    text_block = ''.join(sentences)
    # print(strike_list)
    # ---------------------------
    # 4. Define a list of strikethrough keywords
    #    You can also put full sentences here.
    # ---------------------------
    # strike_list = [
    #     "nanu",      # Strikes any line containing "Lorem"
    #     "Additional line with"   # Strikes any line containing "something"
    # ]

    # ---------------------------
    # 5. Wrap text to fit image width
    # ---------------------------
    max_line_width = 60
    wrapped_lines = textwrap.wrap(text_block, width=max_line_width)
    available_width = img_width - 2 * 50  # subtract some margins (e.g. 50 px each side)
    avg_char_width, _ = draw.textsize("A", font=font)  # measure width of a single char
    max_chars_per_line = max(1, available_width // avg_char_width)

    # Now wrap using that character count
    wrapped_lines = textwrap.wrap(text_block, width=max_chars_per_line)
    strike_list = [random.choice(wrapped_lines)]
    # print(wrapped_lines)
    # print(strike_list)
    # ---------------------------
    # 6. Render text line by line, checking if line contains any strikethrough keywords
    # ---------------------------
    x, y = 50, 50
    line_spacing = 5

    for line in wrapped_lines:
        # If ANY of the items in strike_list is in the line, draw strikethrough
        if any(strike_item in line for strike_item in strike_list):
            draw_text_with_strikethrough(draw, line, font, x, y, fill=0)
        else:
            draw.text((x, y), line, font=font, fill=0)

        # Move down for the next line
        _, top, _, bottom = draw.textbbox((x, y), line, font=font)
        line_height = bottom - top
        y += line_height + line_spacing

        if y > img_height - 50:
            break

    # ---------------------------
    # 7. Save the image (grayscale)
    # ---------------------------
    image.save(r"E:\StrikeThroughData\{}.png".format(id))
    # print("Image saved as black_and_white_text_with_strikethrough_list.png")

    return ''.join(wrapped_lines), strike_list[0] 
# driver(data['text'].unique()[6][:1500])


In [5]:
import warnings
warnings.filterwarnings("ignore")


import uuid
from tqdm import tqdm
def generate_uuids(count=1):
    return [str(uuid.uuid4()) for _ in range(count)]

# Example usage: Generate 5 UUIDs
uuids = generate_uuids(5000)
mapping = []
for i in tqdm(range(len(uuids))):
    text = data['text'].values[i][:3000]
    if len(text) > 2000:
        base_text, strikethrough  = driver(text, uuids[i])
        replaced_text = base_text.replace(strikethrough, "")
        mapping.append([uuids[i]+ 'png', base_text, strikethrough, replaced_text])
        # print(len(base_text), len(replaced_text))
    
    # driver(data['text'].values[i][:1200], uuids[i])
pd.DataFrame(mapping, columns=['ID', 'Complete Text', 'Strike Through', 'Replaced Text']).to_csv(r"E:\StrikeThroughData\Final_dataset.csv")

100%|██████████| 5000/5000 [50:48<00:00,  1.64it/s] 


In [11]:
pd.DataFrame(mapping, columns=['ID', 'Complete Text', 'Strike Through', 'Replaced Text']).to_csv(r"E:\StrikeThroughData\Final_dataset.csv")